# SQLAlchemy One to Many Lab Continued

## Objectives

1.  Create multiple "has many"/"belongs to" relationships with SQLAlchemy
2.  Add data to the database for each table and ensure that the data is properly related to other tables
3.  Query the database containing the "has many"/"belongs to" relationships

## Sports, Players, Teams, Cities
Okay, so, we have just finished creating our first has many / belongs to relationship. Now lets take it a step further and add in another layer, since, domains adn relationships will generally be bigger than just two tables. Let's look at sports teams. One sports team will have many players. But what does our sports team belong to? Specifically, what is a sports team affiliated with that other sports teams may also be affiliated with? A City? That's right! But wait! Can't a team also belong to a specific sport? That's also right! This is analagous to a song belonging to a particular genre, but that can get a bit tricky when we get into songs that are chill electro pop funk -- that's too many genres!!

Back to our sports. Let's build these models out in our **models.py** file. The models below list the attributes each should contain as well as its relationship to other models. Our job is to take this information and create the SQLAlchemy tables.

> **Note:** Because we are going to leverage the relationships between these tables, there is no need for us duplicate information accross tables. For example, players do not need to be instantiated with the name of the team they play for as long as they are related with that team. However, we will want to be able to query a player instance for their team. Be sure to create your tables so that they provide this functionality for all relationships.

* **Players**:
    - Name
    - Number (if applicable)
    - Height
    - Weight
    - *Belongs to a team*
    
* **Teams**:
    - Name
    - *Belongs to a city*    
    - *Belongs to a sport*
    - *Has many players*    

* **Sports**:
    - Name
    - *Has many teams*    

* **City**:
    - Name
    - State
    - *Has many teams*    


## Instructions

1. In the **models.py** file, create the SQLAlchemy schema and for each of our models.
2. The two cities we will be working with are New York, New York, and Los Angeles, California
3. The two sports we will be working with are Basketball and Baseball
4. We have provided for us csv files containing the roster data for the LA Lakers, LA Dodgers, NY Knicks, and NY Yankees. Read these files using pandas to turn this data into objects we can use to populate our tables. Remember to also create the appropriate relationships as we create the class instance objects and commit them to the tables in our data base.
5. Once the data has been created and our relationships are set up, we can go ahead an start working on our queries. Open the **queries.py** file and fill out the functions with the necessary code to meet the deliverables.


In [1]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

Base = declarative_base()


# write the Player, City, Sport and Team tables below
class Player(Base):
    __tablename__ = 'players'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    number = Column(Integer)
    height = Column(Integer)
    weight = Column(Integer)
    age = Column(Integer)
    team_id = Column(Integer, ForeignKey('teams.id'))
    team = relationship('Teams', back_populates = 'roster')

class Sports(Base):
    __tablename__ = 'sports'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    teams = relationship('Teams', back_populates = 'sport')

class City(Base):
    __tablename__ = 'city'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    state = Column(String)
    teams = relationship('Teams', back_populates = 'city')

class Teams(Base):
    __tablename__ = 'teams'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    city_id = Column(Integer, ForeignKey('city.id'))
    city = relationship('City', back_populates = 'teams' )
    sport_id = Column(Integer, ForeignKey('sports.id'))
    sport = relationship('Sports', back_populates = 'teams')
    roster = relationship(Player, back_populates = 'team')



engine = create_engine('sqlite:///sports.db')
Base.metadata.create_all(engine)


In [3]:
from models import *
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sports.db')

Session = sessionmaker(bind = engine)
session = Session()

# below we are reading the csv files to create the data we will need to create the players
# pandas returns a DataFrame object from reading the CSV
# we then tell the DataFrame object to turn each row into dictionaries
# by giving to_dict the argument "orient='records'"
# we are telling our DataFrame to make each row a dictionary using the column headers
# as the keys for the key value pairs in each new dictionary
# feel free to uncomment lines 18-21 to see each step of the process in your terminal
# ____ example ______
# la_dodgers0 = pd.read_csv('la_dodgers_baseball.csv')
# la_dodgers1 = pd.read_csv('la_dodgers_baseball.csv').to_dict()
# la_dodgers2 = pd.read_csv('la_dodgers_baseball.csv').to_dict(orient='records')
# import pdb; pdb.set_trace()
# __________________
la_dodgers = pd.read_csv('la_dodgers_baseball.csv').to_dict(orient='records')
la_lakers = pd.read_csv('la_lakers_basketball.csv').to_dict(orient='records')
ny_yankees = pd.read_csv('ny_yankees_baseball.csv').to_dict(orient='records')
ny_knicks = pd.read_csv('ny_knicks_basketball.csv').to_dict(orient='records')

#Istantiate Cities
ny = City(name = 'New York', state = 'NY')
la = City(name = 'Los Angeles', state = 'CA')
cities = [ny,la]

#Istantiate Sports
basketball = Sports(name = 'Basketball')
baseball = Sports(name = 'Baseball')
sports = [basketball,baseball]

#Istantiate Teams
la_dodgers_team = Teams(name = 'LA Dodgers', city = la, sport = baseball)
la_lakers_team = Teams(name = 'LA Lakers', city = la, sport = basketball)
ny_yankees_team = Teams(name = 'NY Yankees', city = ny, sport = baseball)
ny_knicks_team = Teams(name = 'NY Knicks', city = ny, sport = basketball)
teams = [la_dodgers_team, la_lakers_team, ny_yankees_team, ny_knicks_team]

#Istantiate Players
def heightconvert(height):
    string = ''.join(char for char in height if char.isalnum())
    feet = int(string[0])
    inches = int(string[1:])
    return feet * 12 + inches

def create_players(team_data, team):
    players_per_team = []
    for x in team_data:
        player = Player(name=x.get('name'),age = x.get('age', None),\
        number=x.get('number', None), height= heightconvert(x['height']), \
        weight=x.get('weight', None), team = team)
        players_per_team.append(player)
    return players_per_team

dodgers_roster = create_players(la_dodgers,la_dodgers_team)
lakers_roster = create_players(la_lakers, la_lakers_team)
yankees_roster = create_players(ny_yankees, ny_yankees_team)
knicks_roster = create_players(ny_knicks, ny_knicks_team)

session.add_all(dodgers_roster)
session.add_all(lakers_roster)
session.add_all(yankees_roster)
session.add_all(knicks_roster)
session.add_all(teams)
session.add_all(sports)
session.add_all(cities)

session.commit()

# now that we have the data for each player
# add and commit the players, teams, sports and cities below
# we will need to probably write at least one function to iterate over our data and create the players
# hint: it may be a good idea to creat the Teams, Cities, and Sports first


In [4]:
def return_all_players_in_los_angeles():
    # here we want to return all players that are associated with
    # a sports team in LA
    return session.query(Player).join(Teams).filter(Teams.city == la).all()

In [5]:
vars((return_all_players_in_los_angeles())[0])

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x11558de10>,
 'team_id': 1,
 'weight': 190,
 'number': 75,
 'id': 1,
 'age': None,
 'height': 74,
 'name': 'Scott Alexander'}

In [9]:
def return_tallest_player_in_los_angeles():
    # here we want to return the tallest player associted with
    # a sports team in LA
    la_players = return_all_players_in_los_angeles()
    return max(list(map(lambda player: player.height,la_players)))
#     obj_height_dict = {player:player.height for player in la_players}
#     return max(obj_height_dict, key = obj_height_dict.get)

In [19]:
def return_tallest_player_in_los_angeles():
    tallest_player = session.query(Player,func.max(Player.height)).join(Teams).join(City).filter(City.name=='Los Angeles').all()
    return tallest_player.pop()



In [20]:
return_tallest_player_in_los_angeles()

(<models.Player at 0x1158924a8>, 85)